***
<center><font color="dodgerblue"><font size="7"><b><i>Analyse de Données Développeur<br />(C5-160512-INFO)</i></b></font></font></center>

***

<center><font color="darkorange"><font size="8"><b><i>4) Apprentissage Non Supervisé (TP #1) <br/><font size="6">&copy C. Frélicot, Automne 2023</font></i></b></font></font></center>

***

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.matlib import repmat
import warnings

from scipy.cluster.hierarchy import centroid
from sympy.physics.mechanics.functions import inertia

warnings.filterwarnings('ignore')

***
# 1)  Tableau de données (du TD)
***
## Lecture de fichier formatté à l'aide de *Pandas*.

In [2]:
df = pd.read_excel("MLlogiciel.xls",sheet_name='Feuille1',index_col=0)
row_label = df.index.values 
col_label = df.columns.values
logiciel = df['ML'].values
X = df.select_dtypes(include='number').values
n, p = np.shape(X)
df

,S,A,A+,ML
x,3,13,17,Green
y,12,6,11,Yellow
z,10,14,20,Green
t,1,14,2,Blue
u,7,15,9,Blue


## Variables et quelques points utilisés en TD.

In [3]:
x = df.loc['x'].values[:p]
y = df.loc['y'].values[:p]
S = df['S'].values
A = df['A'].values
Aplus = df['A+'].values

***
# 2) Calcul de Distances
***

## 2-1) Ecrivez une fonction *dist* permetant de calculer la distance de Manhattan (ou Cityblock), euclidienne (*euclidean*, distance par défaut), de Chebychev (ou Chebyshev), et Cosinus, entre deux points $x$ et $y$. 

In [8]:
def dist(x,y,dname='euclidean'):
    if dname == 'manhattan':
        d = np.sum(np.abs(x-y))
    elif dname == 'euclidean':
        d = np.sqrt(np.sum((x-y)**2))
    elif dname == 'chebyshev':
        d = np.max(np.abs(x-y))
    elif dname == 'cosine':
        d = 1 - np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    elif dname == 'correlation':
        d = 1 - np.dot(x-np.mean(x),y-np.mean(y))/(np.linalg.norm(x-np.mean(x))*np.linalg.norm(y-np.mean(y)))
    
    return d

## Testez-la de sorte de retrouver des résultats de TD : 
## - toutes distances entre $x$ et $y$
## - distances *cosinus* entre toutes les variables

In [5]:
print(dist(x,y,'manhattan'),dist(x,y,'euclidean'),dist(x,y,'chebyshev'),dist(x,y,'cosine'))

22 12.884098726725126 9 0.1971677544087882


In [6]:
dist(A,Aplus,'cosine')

0.1489101059217074

## 2-2) Pouvez-vous calculer $d_{correl}(A,A+)$ avec votre fonction ?

In [9]:
dist(A,Aplus,'correlation')


1.0155580791462095

## Il existe bien sûr des fonctions python pour le calcul de distances. Comprenez leur usage. 

In [10]:
from scipy.spatial.distance import pdist, squareform
D2 = pdist(X,'euclidean')**2
print(D2)
print(squareform(D2))
squareform(squareform(D2))

[166.  59. 230.  84. 149. 266. 110. 405. 131.  86.]
[[  0. 166.  59. 230.  84.]
 [166.   0. 149. 266. 110.]
 [ 59. 149.   0. 405. 131.]
 [230. 266. 405.   0.  86.]
 [ 84. 110. 131.  86.   0.]]


array([166.,  59., 230.,  84., 149., 266., 110., 405., 131.,  86.])

## Calculez les distances *cosinus* entre toutes les variables avec la fonction adéquate, puis les distances *corrélation* de sorte de retrouver les résulats obtenus avec votre fonction.

In [13]:
Dcos = pdist(X,'cosine')
Dcor = pdist(X,'correlation')

print(Dcos)
print(Dcor)

[0.19716775 0.03177561 0.28519489 0.09373804 0.07364385 0.52026574
 0.16484858 0.34229301 0.07457967 0.12025733]
[1.38825818 0.06323409 0.69328917 0.53367863 1.04120428 1.99621742
 1.99627096 1.04577342 0.87274305 0.01499483]


***
# 3) Algorithme K-Means 
***

## 3-1) Ecrivez une fonction *prototyping(data,indicator)* de calcul des *barycentres* des clusters d'un tableau de données *data* décrit par une indicatrice de partition. 
## Vous la testerez sur les données du tableau $X$  pour l'indicatrice $Y^{(3)}=(3,2,3,1,1)$ afin de retrouver les résultats du TD.

In [4]:
def prototyping(data,indicator):
        
    unique = np.unique(indicator)
    barycentres = {}
    
    for cluster in unique:
        cluster_points = data[np.array(indicator) == cluster]
        barycentres[cluster] = np.mean(cluster_points,axis=0)
        
    return barycentres

In [5]:
Y3 = [3,2,3,1,1]
prototyping(X,Y3)

{1: array([ 4. , 14.5,  5.5]),
 2: array([12.,  6., 11.]),
 3: array([ 6.5, 13.5, 18.5])}

## 3-2) Ecrivez une fonction *clustering(data,centroids,dname)* d'*affectation* des données d'un tableau *data* à des *clusters* au sens de la distance *dname* (euclidienne par défaut) au plus proche *centroid*.
## Vous la testerez sur les données du tableau $X$ et les *centroids* obtenus à la question précédente afin de retrouver les résultats du TD.

In [6]:
def clustering(data,centroids,dname='euclidean'):

    clusters = []
    distances_to_centroids = []
    
    for point in data:
        distances = {cluster:dist(point,centroid,dname) for cluster,centroid in centroids.items()}
        closest = min(distances,key=distances.get)
        clusters.append(closest)
        distances_to_centroids.append(distances[closest])
        
    return clusters, distances_to_centroids

In [9]:
clustering(X,prototyping(X,Y3))

([3, 2, 3, 1, 1],
 [3.840572873934304,
  0.0,
  3.840572873934304,
  4.636809247747852,
  4.636809247747852])

## Que représente l'enchaînement des deux fonctions en termes d'algorithme d'apprentissage non supervisé ? 

## 3-3) Exécutez séparément plusieurs fois les cellules suivantes afin de comprendre à quoi le code pourrait vous servir.

In [10]:
np.random.permutation(n)

array([0, 4, 1, 2, 3])

In [11]:
c = 3
np.random.permutation(n)[:c]

array([3, 2, 0])

## 3-3) Codez l'algorithme *K-means* dans une fonction *kmeans(data,c,maxiter=50,eps=0.001)* qui appellera les fonctions *clustering* et *prototyping*.  
## L'algorithme doit s'arrêter dès que le nombre d'itérations atteint *maxiter* ou si les *centroids* ne bougent (presque) plus (eps).
## L'initialisation sera réalisée en choisissant aléatoirement *c* points du tableau *data*. 
## Vous la testerez sur les données du tableau *X* du TD.

In [12]:
def kmeans(data,clust_nb,maxiter=10,eps=0.01,dname='euclidean'):

    np.random.seed(42)
    centroids = { i + 1 : data[idx] for i, idx in enumerate(np.random.choice(data.shape[0], clust_nb, replace = False))}
    
    for t in range(maxiter):
        
        indicator, _ = clustering(data,centroids,dname)
        new_centroids = prototyping(data,indicator)
        
        max_shift = max(np.linalg.norm(new_centroids[c] - centroids[c]) for c in centroids)
        centroids = new_centroids
        
        if max_shift < eps:
            break

    return indicator, centroids

In [19]:
Y, C = kmeans(X,3,100,0.001)

Y

[3, 1, 3, 2, 2]

## Si vous ne retrouvez pas $Y^{(3)}=(3,2,3,1,1)$, pensez vous qu'il suffirait d'augmenter *maxiter* ou diminuer *eps* ?

## 3-4) Modifiez votre fonction *kmeans* de sorte qu'elle retourne aussi l'inertie intra-clusters $\mathcal{D}(U,C)$ (ou $\mathcal{D}(Y,C)$ de la partition finale.

In [20]:
def kmeans(data,clust_nb,maxiter=50,eps=0.001,dname='euclidean'):
    
    np.random.seed(42)
    centroids = { i + 1 : data[idx] for i, idx in enumerate(np.random.choice(data.shape[0], clust_nb, replace = False))}
    
    for t in range(maxiter):
        
        indicator, distances = clustering(data,centroids,dname)
        new_centroids = prototyping(data,indicator)
        
        max_shift = max(np.linalg.norm(new_centroids[c] - centroids[c]) for c in centroids)
        centroids = new_centroids
        
        if max_shift < eps:
            break
            
    inertia = sum(d ** 2 for d in distances)

    return indicator, centroids, inertia

## Des exécutions peuvent retourner de bonnes partitions... d'autres moins bonnes. Donnez des exemples. 

In [24]:
Y, C, W = kmeans(X,3,100,1e-6)

W

72.5

## Exécutez le code ci-dessous. Quel est son but ?

In [25]:
w = []
for i in range(1000):
    Y, C, W = kmeans(X,3,100,1e-6)
    w.append(W)
print(min(w),max(w))

72.5 72.5


***
# 4) Clustering de vins italiens décrits par des composants chimiques
***

## Données Réelles : 13 mesures sur 178 vins de 3 viticulteurs différents provenant de la même région https://archive.ics.uci.edu/ml/datasets/wine

In [26]:
from sklearn.datasets import load_wine
data = load_wine()
X = data.data
Y_true = data.target
n, p = np.shape(X)
I = np.eye(p)
D = np.eye(n)/n

## Exécutez votre fonction kmeans de sorte d'obtenir une partition en 3 clusters

In [27]:
Y, C, W = kmeans(X,3,100,1e-6)

## La fonction importée ci-dessous permet de croiser les modalités de deux indicatrices (variable catégorielle). Comment l'utiliser pour valider ou non la partition obtenue ?

In [28]:
from scipy.stats.contingency import crosstab

In [29]:
crosstab(Y,Y_true)

CrosstabResult(elements=(array([1, 2, 3]), array([0, 1, 2])), count=array([[13, 20, 29],
       [46,  1,  0],
       [ 0, 50, 19]]))

## Que pensez-vous du résultat ?